In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import (metrics, model_selection, linear_model, preprocessing, ensemble, neighbors)
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import numpy as np
import pandas as pd
import pprint as pp
import re
#import xgboost as xgb

# import the "crash" data
data = pd.read_csv("my_map_grid.csv",header=7)

# preprocessing
# lowercase: http://stackoverflow.com/a/38931854
data.columns = data.columns.str.lower()
# remove spaces
data.columns = data.columns.str.replace(' ', '_')
# special cases
data.columns = data.columns.str.replace('crash_i_d', 'crash_id')
# remove whateva data

# replace ['No Data','Not Applicable'] with NaN
data.replace(to_replace='No Data', value=np.nan, inplace=True)
data.replace(to_replace='Not Applicable', value=np.nan, inplace=True)

dummies_needed_list = [
 'crash_severity',
 'day_of_week',
 'intersection_related',
 'light_condition',
 'manner_of_collision',
 'number_of_entering_roads',
 'road_base_type',
 'surface_condition' # factorized
        ]
# encode data for dummies_needed_list
for feat in dummies_needed_list:
    data = pd.concat([data,pd.get_dummies(data[feat],prefix=var)],axis=1)
    data = data.drop([feat],axis=1)

#remove data that will not be 
crash_id = data.crash_id
data = data.drop(['crash_id'],axis=1)

In [14]:
data

,average_daily_traffic_amount,average_daily_traffic_year,crash_death_count,crash_incapacitating_injury_count,crash_non-incapacitating_injury_count,crash_not_injured_count,crash_possible_injury_count,crash_time,crash_year,intersecting_street_name,...,surface_condition_Four Entering Roads,surface_condition_Other (Explain In Narrative),surface_condition_Six Entering Roads,surface_condition_Three Entering Roads - T,surface_condition_Three Entering Roads - Y,surface_condition_Traffic Circle,surface_condition_Concrete,surface_condition_Flex Base (Granular),surface_condition_Dry,surface_condition_Wet
0,NaN,NaN,0,0,1,0,0,2023,2015,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,NaN,NaN,0,0,1,2,0,1800,2015,TILLERY ST,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,NaN,NaN,0,0,0,1,1,635,2015,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,NaN,NaN,0,0,1,1,0,1051,2015,AIRPORT BLVD,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,NaN,NaN,0,1,0,1,0,538,2015,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,NaN,NaN,0,0,1,2,0,1600,2015,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,NaN,NaN,0,0,1,0,0,810,2015,WOODROW AVE,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,NaN,NaN,0,0,0,2,0,1047,2015,LIGHTSEY RD,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8,44197,2014,0,1,0,1,0,1525,2015,ARBORETUM BLVD,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
9,NaN,NaN,0,0,1,1,0,1419,2015,SAN JACINTO BLVD,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# convert 'Wet' 'Dry' to '1' '0'
data['surface_condition'] = data['surface_condition'].factorize()[0]
# DOC: rename col http://stackoverflow.com/a/11346337
data.rename(columns={'surface_condition':'surface_wet'})
# print number of unique
for colname in data.columns:
    print("% 4d : %s" % (len(data[colname].unique()), colname))
# remove data which is has no importance
# better to drop cols with all NaN and convert "unimportant" data to NaN
#  - can't universally decide to drop col just based on uniqueness
# e.g. all of object_struck is 'Not Applicable' and useless, but if surface_condition had only one value "dry" this would be important
# ? for colname in data.columns:
# colname = 'object_struck'
# if(len(data[colname].unique()) == 1):
#   print("-I-: dropping %s for having all homogenous values %s", (colname, data[colname].unique()[0]))
#   data.drop(colname,axis=1,inplace=True)


    

print(data.head())
print(data.info())
if(1):
  data.describe()
  data.hist()
  data.corr().plot() # TODO: seaborn
  plt.show()
else:
  print("-I-: Skipping...")

pairplot_var_list = [
# 'crash_id',
 'average_daily_traffic_amount',
 'average_daily_traffic_year',
 'crash_death_count',
# 'crash_incapacitating_injury_count',
# 'crash_non-incapacitating_injury_count',
# 'crash_not_injured_count',
# 'crash_possible_injury_count',
 'crash_severity',
 'crash_time',
 'crash_year',
 'day_of_week',
# 'intersecting_street_name',
 'intersection_related',
# 'latitude',
 'light_condition',
# 'longitude',
 'manner_of_collision',
 'medical_advisory_flag',
 'number_of_entering_roads',
 'number_of_lanes',
# 'object_struck',
 'road_base_type',
 'speed_limit',
# 'street_name',
 'surface_condition'
 ]

# tmp disable
if(0):
    sns.pairplot(data, vars=pairplot_var_list)
    plt.show()

# alternative visualisation
datapt = data.pivot_table(values=['crash_death_count','crash_incapacitating_injury_count','crash_non-incapacitating_injury_count'], index=['speed_limit','crash_time'])
print(datapt)

pp.pprint(list(pd.get_dummies(data[dummies_needed_list]).columns))
pp.pprint(list(pd.get_dummies(data[dummies_needed_list]).columns.str.replace('[,\s]+','_').str.lower()))
'''
 'Dark, Lighted', 'dark_lighted_yes'
 'Dark, Not Lighted', 'dark_lighted_no'
 'Dark, Unknown Lighting', 'dark_lighted_unknown'
 'Dawn',
 'Daylight',
 'Dusk',
 'Unknown',
'''